
Carregando arquivos e a visualização dos dados.



In [27]:
import pandas as pd

# Carregar os dados de treino e teste
train_data = pd.read_csv('/content/ReutersGrain-test.csv')
test_data = pd.read_csv('/content/ReutersGrain-test.csv')

# Visualizar as primeiras linhas dos datasets
train_data.head(), test_data.head()

import pandas as pd

# Listar colunas dos datasets
train_columns = train_data.columns
test_columns = test_data.columns

train_columns, test_columns



(Index(['Text', 'class-att', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
        'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
        ...
        'Unnamed: 172', 'Unnamed: 173', 'Unnamed: 174', 'Unnamed: 175',
        'Unnamed: 176', 'Unnamed: 177', 'Unnamed: 178', 'Unnamed: 179',
        'Unnamed: 180', 'Unnamed: 181'],
       dtype='object', length=182),
 Index(['Text', 'class-att', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
        'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
        ...
        'Unnamed: 172', 'Unnamed: 173', 'Unnamed: 174', 'Unnamed: 175',
        'Unnamed: 176', 'Unnamed: 177', 'Unnamed: 178', 'Unnamed: 179',
        'Unnamed: 180', 'Unnamed: 181'],
       dtype='object', length=182))

Função de pré-processamento

In [28]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string
import re

nltk.download('punkt')
nltk.download('stopwords')

def preprocess(text):
    # Converte o texto para minúsculas
    text = text.lower()
    # Remove números
    text = re.sub(r'\d+', '', text)
    # Remove pontuação
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenização
    tokens = word_tokenize(text)
    # Remoção de stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    # Reune os tokens em uma string
    processed_text = ' '.join(tokens)
    return processed_text

# Aplicar o pré-processamento aos dados de treino e teste
train_data['processed_text'] = train_data['Text'].apply(preprocess)
test_data['processed_text'] = test_data['Text'].apply(preprocess)

# Visualizar os dados processados
train_data.head(), test_data.head()




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(                                                Text  \
 0  'ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN R...   
 1  'CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN ST...   
 2  'JAPAN TO REVISE LONG-TERM ENERGY DEMAND DOWNW...   
 3  'THAI TRADE DEFICIT WIDENS IN FIRST QUARTER Th...   
 4  'INDONESIA SEES CPO PRICE RISING SHARPLY Indon...   
 
                                            class-att  \
 0   businessmen and officials said.\n    They tol...   
 1   the China Daily said.\n    It also said that ...   
 2   ministry officials said.\n    MITI is expecte...   
 3   the Business Economics Department said.\n    ...   
 4                                     Hasrul Harahap   
 
                                           Unnamed: 2  \
 0   in the short-term Tokyo\'s loss might be\nthe...   
 1                                          or 25 pct   
 2   they said.\n    The decision follows the emer...   
 3                                            however   
 4                    junio

Utilizando dois algoritmos de aprendizado de máquina: Naive Bayes e SVM (Support Vector Machine).

In [29]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from tabulate import tabulate

# Vetorização dos textos
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_data['processed_text'])
y_train = train_data['class-att']
X_test = vectorizer.transform(test_data['processed_text'])
y_test = test_data['class-att']

# Treinamento e avaliação do modelo Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)
nb_predictions = nb_model.predict(X_test)
nb_accuracy = accuracy_score(y_test, nb_predictions)
nb_report = classification_report(y_test, nb_predictions, output_dict=True)

# Treinamento e avaliação do modelo SVM
svm_model = SVC()
svm_model.fit(X_train, y_train)
svm_predictions = svm_model.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_predictions)
svm_report = classification_report(y_test, svm_predictions, output_dict=True)

# Criar DataFrames para os relatórios de classificação
nb_report_df = pd.DataFrame(nb_report).transpose()
svm_report_df = pd.DataFrame(svm_report).transpose()

# Exibir as precisões e os relatórios de classificação em tabelas
print("Accuracy Scores")
accuracy_data = {
    "Model": ["Naive Bayes", "SVM"],
    "Accuracy": [nb_accuracy, svm_accuracy]
}
accuracy_df = pd.DataFrame(accuracy_data)
print(tabulate(accuracy_df, headers='keys', tablefmt='psql'))




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

Accuracy Scores
+----+-------------+------------+
|    | Model       |   Accuracy |
|----+-------------+------------|
|  0 | Naive Bayes |   0.157285 |
|  1 | SVM         |   0.206954 |
+----+-------------+------------+


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
